In [4]:
import pandas as pd
import numpy as np


In [5]:
# import cleaned data
df = pd.read_csv('../data/cleaned_tender_data.csv')
df.head()

,id,title,cpv,contract_type,description,country,contracting_authority_type,contracting_authority_activity,value,type_of_procedure,government_procurement_agreement,year,is_framework_agreement,is_dynamic
0,101996-2022,Dobava in namestitev strojne in programske opr...,30200000,Supplies,Predmet javnega naročila je dobava in namestit...,Slovenia,Body governed by public law,Health,NaN,Open procedure,No,2022,Yes,No
1,10001-2022,Elektrosaimniecības dispečerpunktu un apakšsta...,72265000,Services,Elektrosaimniecības dispečerpunktu un apakšsta...,Latvia,NaN,"Urban railway, tramway, trolleybus or bus serv...",950000.0,Open procedure,Yes,2022,Yes,Yes
2,10003-2022,CS-02 Dezvoltare GIS la nivelul noului sistem...,51612000,Services,Servicii de „Dezvoltare GIS la nivelul noului ...,Romania,NaN,Water,7515227.0,Open procedure,No,2022,Yes,Yes
3,10004-2022,Platforma ESB – Enterprise Service BUS,72260000,Services,Achizitia ce are ca obiect Platforma ESB – Ent...,Romania,NaN,"Production, transport and distribution of gas ...",12149000.0,Open procedure,Yes,2022,Yes,Yes
4,10006-2022,Oficina de gobierno y analitica de datos,72212461,Services,Oficina de gobierno y analitica de datos,Spain,NaN,Airport-related activities,2200760.0,Open procedure,No,2022,Yes,Yes


In [11]:
# group by country and show count.
df["country"].value_counts()

Germany        18
Poland         18
Netherlands    10
Czechia        10
Lithuania       8
Spain           6
France          4
Romania         4
Finland         3
Portugal        3
Austria         3
Belgium         2
Estonia         2
Slovenia        2
Sweden          2
Ireland         1
Latvia          1
Slovakia        1
Hungary         1
Malta           1
Croatia         1
Name: country, dtype: int64